# Notebook for INFORMS 2019 Presentation

--- 
**IMPORTANT NOTE:**
- The examples below use 'OSRM-online' as the `dataProvider`.  This data provider has the advantage of not requiring an API key.  However, this is using the OSRM "demo" server.  This means that you're likely to encounter errors if the server is overloaded.
- If you get an error indicating that "OSRM is not available", I suggest that you switch to a different data provider.  See the following link for data provider options:
    - https://veroviz.org/docs/dataproviders.html
    - 'ORS-online' is the recommended data provider
    
---

Before continuing, please check out the installation instructions:
- https://veroviz.org/documentation.html

---

This notebook uses the following functions, each of which has extensive documentation (with examples):
- `generateNodes()` - https://veroviz.org/docs/veroviz.generateNodes.html#veroviz.generateNodes.generateNodes
- `createLeaflet()` - https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.createLeaflet
- `addLeafletCircle()` - https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletCircle
- `addLeafletMarker()` - https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletMarker
- `getTimeDist2D()` - https://veroviz.org/docs/veroviz.getTimeDist2D.html#module-veroviz.getTimeDist2D
- `getTimeDist3D()` - https://veroviz.org/docs/veroviz.getTimeDist3D.html#module-veroviz.getTimeDist3D
- `createArcsFromNodeSeq()` - https://veroviz.org/docs/veroviz.generateArcs.html#veroviz.generateArcs.createArcsFromNodeSeq
- `getShapepoints2D()` - https://veroviz.org/docs/veroviz.getShapepoints2D.html#module-veroviz.getShapepoints2D
- `getShapepoints3D()` - https://veroviz.org/docs/veroviz.getShapepoints3D.html#module-veroviz.getShapepoints3D
- `addStaticAssignment()` - https://veroviz.org/docs/veroviz.buildAssignments.html#veroviz.buildAssignments.addStaticAssignment
- `createCesium()` - https://veroviz.org/docs/veroviz.createCesium.html#module-veroviz.createCesium

---

In [1]:
# Import the veroviz package, and make sure
# we're running the latest version:
import veroviz as vrv
vrv.checkVersion()

'Your current installed version of veroviz is 0.2.1. You are up-to-date with the latest available version.'

## Generate test problems

Our first task will be to create a collection of 8 nodes:
- Node 0 will be the depot.  Its location should be generated from within a boundary region.  We'll color this node blue.
- Nodes 1-7 will be the customers.  They should be generated from a normal distribution, centered at the location of the INFORMS 2019 Annual Meeting, with a standard deviation of 1 mile.  These nodes will be red. 

In [2]:
# Define the region within which the depot node will be generated:
depotBoundary = [[47.66, -122.26], 
                 [47.52, -122.25], 
                 [47.50, -122.14], 
                 [47.71, -122.14]]

In [3]:
# Generate 1 depot node.
myNodes = vrv.generateNodes(numNodes        = 1,
                            startNode       = 0,
                            nodeType        = 'depot',
                            nodeName        = 'Depot',
                            leafletIconType = 'star',
                            nodeDistrib     = 'uniformBB',
                            nodeDistribArgs = {'boundingRegion': depotBoundary},
                            snapToRoad      = True,
                            dataProvider    = 'OSRM-online')

- See https://veroviz.org/docs/veroviz.generateNodes.html#veroviz.generateNodes.generateNodes for documentation on the `generateNodes()` function.

In [4]:
# We'll generate our customer nodes via a normal distribution.
# The center point will be the Washington State Convention Center (venue for INFORMS 2019).
# The standard deviation should be expressed in units of meters.
WashStConvCtr = [47.6114, -122.3323]
radiusMeters  = vrv.convertDistance(2, 'mi', 'meters')

In [5]:
# Generate 7 customers nodes.  
# We'll assign these to the category of 'customer', so we can filter 
# for these nodes later if we wish.  Each node will be uniquely named 
# as 'Cust x', where `x` will be replaced by the numbers 1 thru 7.
myNodes = vrv.generateNodes(initNodes       = myNodes,
                            numNodes        = 7,
                            startNode       = 1,
                            nodeType        = 'customer',
                            nodeName        = 'Cust ',
                            incrementName   = True, 
                            leafletColor    = 'red',
                            leafletIconType = 'home',
                            cesiumColor     = 'Cesium.Color.RED',
                            nodeDistrib     = 'normal',
                            nodeDistribArgs = {'center': WashStConvCtr,
                                               'stdDev': radiusMeters},
                            snapToRoad      = True,
                            dataProvider    = 'OSRM-online')

In [6]:
# Display our nodes dataframe, which now contains the depot and 7 customers.
myNodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,0,47.583744,-122.159191,0,Depot,depot,glyphicon,star,blue,0,pin,Cesium.Color.BLUE,0
1,1,47.593616,-122.390207,0,Cust 1,customer,glyphicon,home,red,1,pin,Cesium.Color.RED,1
2,2,47.627642,-122.333594,0,Cust 2,customer,glyphicon,home,red,2,pin,Cesium.Color.RED,2
3,3,47.623066,-122.329148,0,Cust 3,customer,glyphicon,home,red,3,pin,Cesium.Color.RED,3
4,4,47.625333,-122.309323,0,Cust 4,customer,glyphicon,home,red,4,pin,Cesium.Color.RED,4
5,5,47.602023,-122.339865,0,Cust 5,customer,glyphicon,home,red,5,pin,Cesium.Color.RED,5
6,6,47.606760,-122.331306,0,Cust 6,customer,glyphicon,home,red,6,pin,Cesium.Color.RED,6
7,7,47.595113,-122.386955,0,Cust 7,customer,glyphicon,home,red,7,pin,Cesium.Color.RED,7


- See https://veroviz.org/docs/nodes.html for documentation of the 'nodes' dataframe structure.

## Visualize Nodes on a Map
The following cells describe how to display information about the distributions from which our nodes were drawn.  We'll also display the nodes themselves on the map.

In [7]:
# Let's begin by visualizing the distribution parameters.

# Start with an empty map and draw the depot boundary polygon:
myMap = vrv.createLeaflet(boundingRegion = depotBoundary)

# Now, draw some circles to represent the center and spread of the normal 
# distribution from which our customer locations are drawn.
# Each circle will be centered at the Convention Center. 
# We'll show circles of 1, 2, and 3 standard deviations.
myMap = vrv.addLeafletCircle(mapObject = myMap,          center    = WashStConvCtr,
                             radius    = 3*radiusMeters, fillColor = 'red')
myMap = vrv.addLeafletCircle(mapObject = myMap,          center    = WashStConvCtr,
                             radius    = 2*radiusMeters, fillColor = 'green')
myMap = vrv.addLeafletCircle(mapObject = myMap,          center    = WashStConvCtr,
                             radius    = 1*radiusMeters, fillColor = 'blue')

# Use a circle marker to denote the center point:
myMap = vrv.addLeafletMarker(mapObject = myMap,          center    = WashStConvCtr)


# Add some text labels to help explain the map:
myMap = vrv.addLeafletText(mapObject   = myMap, 
                           anchorPoint = WashStConvCtr, 
                           text        = "INFORMS", 
                           fontSize    = 12, fontColor = 'white')
myMap = vrv.addLeafletText(mapObject   = myMap, 
                           anchorPoint = [47.6598, -122.148], 
                           text        = "1 Depot",
                           fontSize    = 12, fontColor = 'darkred',
                           horizAlign  = 'right')
myMap = vrv.addLeafletText(mapObject   = myMap, 
                           anchorPoint = vrv.pointInDistance2D(WashStConvCtr, 180, radiusMeters), 
                           text        = "1 std. dev.",
                           fontSize    = 12, fontColor = 'black')
myMap = vrv.addLeafletText(mapObject   = myMap, 
                           anchorPoint = vrv.pointInDistance2D(WashStConvCtr, 180, 3*radiusMeters), 
                           text        = "3 std. devs.",
                           fontSize    = 12, fontColor = 'black')
myMap = vrv.addLeafletText(mapObject   = myMap, 
                           anchorPoint = vrv.pointInDistance2D(WashStConvCtr, 0, 2.5*radiusMeters), 
                           text        = "7 Customers",
                           fontSize    = 12, fontColor = 'darkred')
myMap

In [8]:
# Now let's add our nodes to the map:
vrv.createLeaflet(mapObject     = myMap,
                  mapBackground = 'cartodb positron', 
                  nodes         = myNodes)

- We can produce maps of just the nodes (without all of the distribution graphics).
- The following examples also show some of the available map backgrounds.  See https://veroviz.org/docs/leaflet_style.html#leaflet-map-background-map-tiles for a complete list of options.

In [9]:
vrv.createLeaflet(mapBackground = 'cartodb dark_matter', 
                  nodes         = myNodes)

In [10]:
vrv.createLeaflet(mapBackground = 'Arcgis Aerial', 
                  nodes         = myNodes)

In [11]:
vrv.createLeaflet(mapBackground = 'Open Topo', 
                  nodes         = myNodes)

In [12]:
vrv.createLeaflet(mapBackground = 'Arcgis Ocean', 
                  nodes         = myNodes)

Detailed documentation of the Leaflet-related functions may be found here:
- `createLeaflet()` - https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.createLeaflet
- `addLeafletCircle()` - https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletCircle
- `addLeafletMarker()` - https://veroviz.org/docs/veroviz.createLeaflet.html#veroviz.createLeaflet.addLeafletMarker

## Travel Matrices

Now that we've created some nodes, our next task is to find the time and distance to travel between each pair of nodes.

We'll want to do this for two different types of vehicles:
1. Trucks, which should travel along a road network and obey speed limits, and 
2. Drones, which won't follow the road, but will have a vertical component to their travel.

In [13]:
# Find the truck's travel time (in seconds) and distance (in meters) 
# between all pairs of nodes.
# We're looking for the fastest time.  Alternatively, we could look
# for the shortest distance.
# We'll need a data provider for this information.
[timeSec, distMeters] = vrv.getTimeDist2D(nodes        = myNodes, 
                                          routeType    = 'fastest',
                                          dataProvider = 'OSRM-online')

In [14]:
# Let's find the travel time from node 1 to node 3:
timeSec[1, 3]

1113.8

In [15]:
# Similarly, we can find the distance from node 1 to node 3:
distMeters[1,3]

16058.4

In [16]:
# `timeSec` is a dictionary.  
# We could print it, but it's difficult to read:
timeSec

{(0, 0): 0.0,
 (0, 1): 1266.7,
 (0, 2): 1027.4,
 (0, 3): 1063.6,
 (0, 4): 1178.8,
 (0, 5): 1458.8,
 (0, 6): 840.1,
 (0, 7): 1241.3,
 (1, 0): 1370.4,
 (1, 1): 0.0,
 (1, 2): 1077.6,
 (1, 3): 1113.8,
 (1, 4): 1276.9,
 (1, 5): 1209.4,
 (1, 6): 937.7,
 (1, 7): 23.3,
 (2, 0): 1069.2,
 (2, 1): 974.9,
 (2, 2): 0.0,
 (2, 3): 247.7,
 (2, 4): 592.6,
 (2, 5): 1188.5,
 (2, 6): 524.5,
 (2, 7): 949.5,
 (3, 0): 970.5,
 (3, 1): 876.2,
 (3, 2): 248.4,
 (3, 3): 0.0,
 (3, 4): 364.2,
 (3, 5): 1089.8,
 (3, 6): 425.8,
 (3, 7): 850.8,
 (4, 0): 1237.4,
 (4, 1): 1246.3,
 (4, 2): 601.0,
 (4, 3): 356.6,
 (4, 4): 0.0,
 (4, 5): 1519.7,
 (4, 6): 562.2,
 (4, 7): 1220.9,
 (5, 0): 1125.0,
 (5, 1): 763.8,
 (5, 2): 757.5,
 (5, 3): 793.7,
 (5, 4): 896.4,
 (5, 5): 0.0,
 (5, 6): 520.2,
 (5, 7): 738.4,
 (6, 0): 866.6,
 (6, 1): 844.9,
 (6, 2): 468.9,
 (6, 3): 505.1,
 (6, 4): 668.2,
 (6, 5): 876.4,
 (6, 6): 0.0,
 (6, 7): 819.5,
 (7, 0): 1347.1,
 (7, 1): 25.4,
 (7, 2): 1054.3,
 (7, 3): 1090.5,
 (7, 4): 1253.6,
 (7, 5): 1186.1,


In [17]:
# Instead, we can convert this dictionary to a dataframe 
# so we can view the travel times in matrix form:
vrv.convertMatricesDictionaryToDataframe(timeSec)

,0,1,2,3,4,5,6,7
0,0.0,1266.7,1027.4,1063.6,1178.8,1458.8,840.1,1241.3
1,1370.4,0.0,1077.6,1113.8,1276.9,1209.4,937.7,23.3
2,1069.2,974.9,0.0,247.7,592.6,1188.5,524.5,949.5
3,970.5,876.2,248.4,0.0,364.2,1089.8,425.8,850.8
4,1237.4,1246.3,601.0,356.6,0.0,1519.7,562.2,1220.9
5,1125.0,763.8,757.5,793.7,896.4,0.0,520.2,738.4
6,866.6,844.9,468.9,505.1,668.2,876.4,0.0,819.5
7,1347.1,25.4,1054.3,1090.5,1253.6,1186.1,914.4,0.0


In [18]:
# Similarly, we can look at a matrix of travel distances:
vrv.convertMatricesDictionaryToDataframe(distMeters)

,0,1,2,3,4,5,6,7
0,0.0,22700.9,17493.2,19305.5,16501.1,16029.5,14801.2,22403.5
1,22940.8,0.0,14246.1,16058.4,14970.8,9904.4,11570.7,297.4
2,17799.1,14099.4,0.0,967.7,4697.6,4734.5,3700.0,13801.9
3,16839.6,13139.9,964.9,0.0,2583.6,3775.0,2740.5,12842.5
4,19737.6,15314.5,3556.4,2606.0,0.0,6014.4,3213.3,15017.0
5,16325.9,9309.1,4235.9,6048.2,5221.5,0.0,1303.1,9011.6
6,15071.2,11356.0,3309.8,5122.1,4034.4,1787.1,0.0,11058.5
7,22643.4,297.4,13948.7,15761.0,14673.3,9607.0,11273.2,0.0


For the drones, we'll need to calculate the travel matrices differently, as they travel in 3 dimensions.  In particular, we'll make the following assumptions:
- The drones will takeoff and land vertically, and 
- They will travel horizontally at a constant altitude

In [19]:
# The `getTimeDist3D` function returns 3 dictionaries.
# - timeSec3D will contain the travel times (in seconds) between each pair of nodes.
# - groundDist will describe only the ground distance between each pair of nodes 
#   (i.e., without any vertical travel).
# - distMeters3D will contain the total distance traveled between each pair of nodes
#   (i.e., both the horizontal and vertical travel).
[timeSec3D, groundDist, distMeters3D] = vrv.getTimeDist3D(
    nodes              = myNodes, 
    routeType          = 'square',
    takeoffSpeedMPS    = vrv.convertSpeed(20, 'mi', 'hr', 'm', 's'),
    landSpeedMPS       = vrv.convertSpeed(5, 'mi', 'hr', 'm', 's'),
    cruiseAltMetersAGL = 100, 
    cruiseSpeedMPS     = vrv.convertSpeed(50, 'mi', 'hr', 'm', 's'))

In [20]:
vrv.convertMatricesDictionaryToDataframe(distMeters3D)

,0,1,2,3,4,5,6,7
0,0.000000,17610.863647,14192.552302,13707.112089,12399.293830,13939.728102,13394.801908,17377.954081
1,17610.863647,0.000000,5894.672924,5838.960866,7229.858430,4099.575128,4864.213535,495.838910
2,14192.552302,5894.672924,0.000000,808.708765,2042.242263,3087.144799,2528.083299,5601.442280
3,13707.112089,5838.960866,808.708765,0.000000,1711.326257,2674.473452,2020.191774,5543.200157
4,12399.293830,7229.858430,2042.242263,1711.326257,0.000000,3662.536982,2844.879735,6934.809678
5,13939.728102,4099.575128,3087.144799,2674.473452,3662.536982,0.000000,1031.616933,3823.661658
6,13394.801908,4864.213535,2528.083299,2020.191774,2844.879735,1031.616933,0.000000,4580.527664
7,17377.954081,495.838910,5601.442280,5543.200157,6934.809678,3823.661658,4580.527664,0.000000


In [21]:
# Here's the total distance to travel from node 1 to node 3:
distMeters3D[1, 3]

# This includes 100 meters for takeoff and 100 meters to land.

5838.960865670521

In [22]:
# The grond distance ignores the change in altitude:
groundDist[1, 3]

5638.960865670521

## Creating a Solution (this would be your solver)
VeRoViz isn't a solver...it's a toolkit for visualizing solutions generated by **your** solver.

In this section we provide some sample code to mimic a solver.  

In [23]:
# We'll need pandas to concatenate dataframes
import pandas as pd

We're going to define a function that will produce routes for our truck and drones.

There's no algorithm here...just a hard-coded assignment of routes. 
The important thing in this function is that it returns some information which 
we'll convert for use by VeRoViz.

Although the "solver" logic is lousy, hopefully it will help you understand one approach for converting your solver's output into a form usable by VeRoViz.

In [24]:
def BLACK_BOX_SOLVER(myNodes, timeSec, timeSec3D):
    # THIS IS A VERY NAIVE "SOLUTION" TO THE PROBLEM OF 
    # VISITING CUSTOMERS WITH BOTH A TRUCK AND MULTIPLE DRONES.
    #
    # THIS FUNCTION IS PROVIDED SIMPLY FOR DEMONSTRATION 
    # PURPOSES. 

    # ------------------------------------------------------
    # Assumptions:
    # - The depot is node 0.  
    # - The truck originates/terminates at the depot.
    # - The truck has a fixed route (defined below).
    # - The drones also have fixed routes (also below).
    # - This "solver" requires exactly 1 depot (node 0) 
    #   and 7 customers (nodes 1-7).
    # - Customer deliveries require 30 seconds (for both 
    #   the truck and drones).
    # - Drones must wait for the truck to land.
    # - Drone launches and recoveries happen instantaneously.
    # - The truck can launch/recovery multiple UAVs 
    #   simultaneously.
    # - Drones have unlimited endurance, and can delivery
    #   any of the packages.
    # ------------------------------------------------------

    # ------------------------------------------------------
    # Here are hard-coded routes for the truck and 2 drones:
    # ------------------------------------------------------
    truckRoute    = [0, 1, 3, 5, 0]
    droneSorties = {
        1: [[1, 2, 3], [3, 4, 5]],
        2: [[1, 6, 5], [5, 7, 0]]
    }

    serviceTime = 30  # seconds
    
    styles = {'truck': {'color': 'blue', 'line': 'solid'},
              'drone1': {'color': 'red', 'line': 'dashed'},
              'drone2': {'color': 'purple', 'line': 'dotted'}
             }
    
    # ------------------------------------------------------
    # Generate an "arcs" dataframe to visualize 
    # the sequence of visits (without any timing details).
    # ------------------------------------------------------
    # Truck - blue route
    myArcs = vrv.createArcsFromNodeSeq(nodeSeq = truckRoute,
                                       nodes   = myNodes, 
                                       leafletColor = styles['truck']['color'],
                                       leafletStyle = styles['truck']['line'])
    # Drone 1 - red dashed routes
    for sortie in droneSorties[1]:
        myArcs = vrv.createArcsFromNodeSeq(initArcs     = myArcs,
                                           nodeSeq      = sortie,
                                           nodes        = myNodes,
                                           leafletColor = styles['drone1']['color'],
                                           leafletStyle = styles['drone1']['line'], 
                                           useArrows    = False)
    # Drone 2 - purple dotted routes
    for sortie in droneSorties[2]:
        myArcs = vrv.createArcsFromNodeSeq(initArcs     = myArcs,
                                           nodeSeq      = sortie,
                                           nodes        = myNodes,
                                           leafletColor = styles['drone2']['color'],
                                           leafletStyle = styles['drone2']['line'], 
                                           useArrows    = False)


    # ------------------------------------------------------
    # Generate a detailed solution, including the timing
    # of visits for each vehicle.
    # ------------------------------------------------------
    solution = {}

    solution['truck'] = []
    solution['drone1'] = []
    solution['drone2'] = []

    # We'll need to figure out when the truck can leave each node
    # (after waiting for drones).
    maxDepartureTime = {}
    for i in truckRoute:
        maxDepartureTime[i] = 0.0

    # Start building the detailed solution, with timing:
    truckStartTime = 0.0    
    for index in list(range(0, len(truckRoute)-1)):
        truckStartNode = truckRoute[index]
        truckEndNode = truckRoute[index+1]
        truckArrivalTime = truckStartTime + timeSec[truckStartNode, truckEndNode]
        truckEndServiceTime = truckArrivalTime + serviceTime

        maxDepartureTime[truckEndNode] = max(maxDepartureTime[truckEndNode], 
                                             truckEndServiceTime)

        for drone in droneSorties:
            for sortie in droneSorties[drone]:
                if (sortie[0] == truckStartNode):

                    # i to j
                    droneStartTime = truckStartTime
                    droneArrivalTime = droneStartTime + timeSec3D[sortie[0], sortie[1]]
                    droneEndServiceTime = droneArrivalTime + serviceTime
                    solution['drone%d' % (drone)].append({
                        'startNode': sortie[0],
                        'startTime': droneStartTime,
                        'status': 'delivery',
                        'modelFile': 'veroviz/models/drone_package.gltf',
                        'arcColor': styles['drone%d' % (drone)]['color'],
                        'arcStyle': styles['drone%d' % (drone)]['line'],
                        'endNode': sortie[1],
                        'droneArrivalTime': droneArrivalTime,
                        'truckArrivalTime': -1,
                        'serviceStartTime': droneArrivalTime,
                        'serviceEndTime': droneEndServiceTime,
                        'endTime': droneEndServiceTime,
                        'packageFile': 'veroviz/models/box_yellow.gltf'
                    })

                    # j to k
                    droneStartTime = droneEndServiceTime
                    droneArrivalTime = droneStartTime + timeSec3D[sortie[1], sortie[2]]
                    droneEndServiceTime = droneArrivalTime
                    solution['drone%d' % (drone)].append({
                        'startNode': sortie[1],
                        'startTime': droneStartTime,

                        'status': 'empty',
                        'modelFile': 'veroviz/models/drone.gltf',
                        'arcColor': styles['drone%d' % (drone)]['color'],
                        'arcStyle': styles['drone%d' % (drone)]['line'],
                        'endNode': sortie[2],
                        'droneArrivalTime': droneArrivalTime,
                        'truckArrivalTime': -1,   # still unknown
                        'serviceStartTime': droneArrivalTime,
                        'serviceEndTime': droneEndServiceTime,
                        'endTime': droneEndServiceTime,
                        'packageFile': None
                    })

                    maxDepartureTime[sortie[2]] = max(maxDepartureTime[sortie[2]], 
                                                    droneEndServiceTime)

        solution['truck'].append({
            'startNode': truckStartNode,
            'startTime': truckStartTime,
            'status': 'delivery' if truckEndNode != 0 else 'empty',
            'modelFile': 'veroviz/models/ub_truck.gltf',
            'arcColor': styles['truck']['color'],
            'arcStyle': styles['truck']['line'],
            'endNode': truckEndNode,
            'droneArrivalTime': -1, # ignored
            'truckArrivalTime': truckArrivalTime,
            'serviceStartTime': truckArrivalTime,
            'serviceEndTime': truckEndServiceTime,
            'endTime': maxDepartureTime[truckEndNode],
            'packageFile': 'veroviz/models/box_blue.gltf' if truckEndNode != 0 else None
        })

        truckStartTime = maxDepartureTime[truckEndNode]

    # Now, update the solution for each drone.
    # When will the truck be there to pick it up?
    for drone in droneSorties:
        for index in list(range(0, len(solution['drone%d' % (drone)]))):
            droneEndNode = solution['drone%d' % (drone)][index]['endNode']
            if (solution['drone%d' % (drone)][index]['status'] == 'empty'):
                solution['drone%d' % (drone)][index]['truckArrivalTime'] = maxDepartureTime[droneEndNode]

    return (myArcs, solution)

In [25]:
# Let's run our dummy solver:
[myArcs, mySolution] = BLACK_BOX_SOLVER(myNodes, timeSec, timeSec3D)

# The warnings here are to be expected.  
# VeRoViz will automatically assign unique odIDs (origin/destination) for each arc.

## Converting Solution into VeRoViz Dataframes

We have now obtained two outputs from our dummy solver:
1. `myArcs` describes just the sequence of node visits for each vehicle.
    - It does not contain any information about the timing of each visit, nor does it describe the turn-by-turn movement of vehicles along these arcs.
    - However, this information can still be useful to get an overall sense of the routes.
2. `mySolution` provides detailed information about the vehicle movements. 
    - This includes the time that vehicles start from, and end at, each origin/destination pair.
    - It also includes any time that a drone waits for a truck (or vice versa).
    - Additionally, each origin/destination pair is decomposed into smaller segments.  For the truck, these segments represent turn-by-turn travel along the road network.  For the drones, this includes takeoff and landing.

In [26]:
# Display our arcs dataframe.
myArcs

,odID,startLat,startLon,endLat,endLon,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,47.583744,-122.159191,47.593616,-122.390207,blue,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
1,2,47.593616,-122.390207,47.623066,-122.329148,blue,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
2,3,47.623066,-122.329148,47.602023,-122.339865,blue,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
3,4,47.602023,-122.339865,47.583744,-122.159191,blue,3,solid,0.8,True,Cesium.Color.ORANGE,3,solid,0.8
4,5,47.593616,-122.390207,47.627642,-122.333594,red,3,dashed,0.8,False,Cesium.Color.ORANGE,3,solid,0.8
5,6,47.627642,-122.333594,47.623066,-122.329148,red,3,dashed,0.8,False,Cesium.Color.ORANGE,3,solid,0.8
6,7,47.623066,-122.329148,47.625333,-122.309323,red,3,dashed,0.8,False,Cesium.Color.ORANGE,3,solid,0.8
7,8,47.625333,-122.309323,47.602023,-122.339865,red,3,dashed,0.8,False,Cesium.Color.ORANGE,3,solid,0.8
8,9,47.593616,-122.390207,47.606760,-122.331306,purple,3,dotted,0.8,False,Cesium.Color.ORANGE,3,solid,0.8
9,10,47.606760,-122.331306,47.602023,-122.339865,purple,3,dotted,0.8,False,Cesium.Color.ORANGE,3,solid,0.8


- Note:  "arcs" dataframes have a specific pre-defined structure within VeRoViz.  See https://veroviz.org/docs/arcs.html for details.

In [27]:
# Take a look at the straight-line arcs and nodes:
vrv.createLeaflet(arcs = myArcs, nodes = myNodes)

In [28]:
# `mySolution` is a dictionary.  
# We can look at this dictionary, but it's not easily readable:
mySolution

{'truck': [{'startNode': 0,
   'startTime': 0.0,
   'status': 'delivery',
   'modelFile': 'veroviz/models/ub_truck.gltf',
   'arcColor': 'blue',
   'arcStyle': 'solid',
   'endNode': 1,
   'droneArrivalTime': -1,
   'truckArrivalTime': 1266.7,
   'serviceStartTime': 1266.7,
   'serviceEndTime': 1296.7,
   'endTime': 1296.7,
   'packageFile': 'veroviz/models/box_blue.gltf'},
  {'startNode': 1,
   'startTime': 1296.7,
   'status': 'delivery',
   'modelFile': 'veroviz/models/ub_truck.gltf',
   'arcColor': 'blue',
   'arcStyle': 'solid',
   'endNode': 3,
   'droneArrivalTime': -1,
   'truckArrivalTime': 2410.5,
   'serviceStartTime': 2410.5,
   'serviceEndTime': 2440.5,
   'endTime': 2440.5,
   'packageFile': 'veroviz/models/box_blue.gltf'},
  {'startNode': 3,
   'startTime': 2440.5,
   'status': 'delivery',
   'modelFile': 'veroviz/models/ub_truck.gltf',
   'arcColor': 'blue',
   'arcStyle': 'solid',
   'endNode': 5,
   'droneArrivalTime': -1,
   'truckArrivalTime': 3530.3,
   'serviceSta

In [29]:
# Instead, it might be easier to view the solution as a dataframe, 
# looking at one vehicle at a time:
pd.DataFrame(mySolution['truck'])

,startNode,startTime,status,modelFile,arcColor,arcStyle,endNode,droneArrivalTime,truckArrivalTime,serviceStartTime,serviceEndTime,endTime,packageFile
0,0,0.0,delivery,veroviz/models/ub_truck.gltf,blue,solid,1,-1,1266.7,1266.7,1296.7,1296.7,veroviz/models/box_blue.gltf
1,1,1296.7,delivery,veroviz/models/ub_truck.gltf,blue,solid,3,-1,2410.5,2410.5,2440.5,2440.5,veroviz/models/box_blue.gltf
2,3,2440.5,delivery,veroviz/models/ub_truck.gltf,blue,solid,5,-1,3530.3,3530.3,3560.3,3560.3,veroviz/models/box_blue.gltf
3,5,3560.3,empty,veroviz/models/ub_truck.gltf,blue,solid,0,-1,4685.3,4685.3,4715.3,4715.3,None


In [30]:
pd.DataFrame(mySolution['drone1'])

,startNode,startTime,status,modelFile,arcColor,arcStyle,endNode,droneArrivalTime,truckArrivalTime,serviceStartTime,serviceEndTime,endTime,packageFile
0,1,1296.700000,delivery,veroviz/models/drone_package.gltf,red,dashed,2,1607.396590,-1.0,1607.396590,1637.396590,1637.396590,veroviz/models/box_yellow.gltf
1,2,1637.396590,empty,veroviz/models/drone.gltf,red,dashed,3,1720.553059,2440.5,1720.553059,1720.553059,1720.553059,None
2,3,2440.500000,delivery,veroviz/models/drone_package.gltf,red,dashed,4,2564.038525,-1.0,2564.038525,2594.038525,2594.038525,veroviz/models/box_yellow.gltf
3,4,2594.038525,empty,veroviz/models/drone.gltf,red,dashed,5,2804.871950,3560.3,2804.871950,2804.871950,2804.871950,None


- NOTE:  The structure of `mySolution` isn't specific to VeRoViz.  Your solver does **not** need to produce an output that looks exactly like this.  
    - In the code below, we'll convert this information into a VeRoViz "assignments" dataframe, which **does** have a clearly-defined structure.
    - See https://veroviz.org/docs/assignments.html for details on "assignments" dataframes.

In [31]:
# Initialize an empty "assignments" dataframe.  
assignmentsDF = vrv.initDataframe('assignments')

for v in mySolution:
    for leg in mySolution[v]:
        # Vehicle moves between two different locations:
        if (v == 'truck'):
            # Movement - 2D
            myShapepoints = vrv.getShapepoints2D(
                objectID       = v,
                modelFile      = leg['modelFile'],
                startLoc       = list(myNodes[myNodes['id'] == leg['startNode']][['lat', 'lon']].values[0]),
                endLoc         = list(myNodes[myNodes['id'] == leg['endNode']][['lat', 'lon']].values[0]),
                startTimeSec   = leg['startTime'],
                expDurationSec = leg['truckArrivalTime'] - leg['startTime'],
                routeType      = 'fastest',
                dataProvider   = 'OSRM-online',
                leafletColor   = leg['arcColor'],
                leafletStyle   = leg['arcStyle'],
                cesiumColor    = 'Cesium.Color.%s' % (leg['arcColor'].upper()),
                cesiumStyle    = leg['arcStyle'])            
        else:
            # movement - 3D            
            myShapepoints = vrv.getShapepoints3D(
                objectID           = v,
                modelFile          = leg['modelFile'],
                startLoc           = list(myNodes[myNodes['id'] == leg['startNode']][['lat', 'lon']].values[0]),
                endLoc             = list(myNodes[myNodes['id'] == leg['endNode']][['lat', 'lon']].values[0]),
                startTimeSec       = leg['startTime'],
                routeType          = 'square',
                takeoffSpeedMPS    = vrv.convertSpeed(20, 'mi', 'hr', 'm', 's'),
                landSpeedMPS       = vrv.convertSpeed(5, 'mi', 'hr', 'm', 's'),
                cruiseAltMetersAGL = 100, 
                cruiseSpeedMPS     = vrv.convertSpeed(50, 'mi', 'hr', 'm', 's'),
                earliestLandTime   = leg['truckArrivalTime'],
                loiterPosition     = 'arrivalAtAlt',
                leafletColor       = leg['arcColor'],
                leafletStyle       = leg['arcStyle'],
                cesiumColor        = 'Cesium.Color.%s' % (leg['arcColor'].upper()),
                cesiumStyle        = leg['arcStyle']) 

        assignmentsDF = pd.concat([assignmentsDF, myShapepoints], 
                                  ignore_index=True, sort=False)        

        # Vehicle is static (service or waiting for drone):
        if (leg['endTime'] > max(leg['droneArrivalTime'], leg['truckArrivalTime'])):
            # static
            assignmentsDF = vrv.addStaticAssignment(
                initAssignments = assignmentsDF, 
                objectID        = v,
                modelFile       = leg['modelFile'],
                loc             = list(myNodes[myNodes['id'] == leg['endNode']][['lat', 'lon']].values[0]),
                startTimeSec    = max(leg['droneArrivalTime'], leg['truckArrivalTime']),
                endTimeSec      = leg['endTime'])

        # Leave a package at the customer:
        if (leg['status'] == 'delivery'):
            assignmentsDF = vrv.addStaticAssignment(
                initAssignments = assignmentsDF, 
                objectID        = 'package %d' % leg['endNode'],
                modelFile       = leg['packageFile'], 
                modelMinPxSize  = 45,
                loc             = list(myNodes[myNodes['id'] == leg['endNode']][['lat', 'lon']].values[0]),
                startTimeSec    = leg['endTime'],
                endTimeSec      = -1)   
            

In [32]:
# Here's our VeRoViz "assignments" dataframe.
# Note that it has many more rows (and a few more columns) than the "arcs" dataframe.
assignmentsDF

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,truck,/veroviz/models/ub_truck.gltf,100,75,0.000000,47.583744,-122.159191,0,31.1967,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
1,1,truck,/veroviz/models/ub_truck.gltf,100,75,31.196678,47.583823,-122.166504,0,42.1065,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
2,1,truck,/veroviz/models/ub_truck.gltf,100,75,42.106472,47.582254,-122.167583,0,44.2749,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
3,1,truck,/veroviz/models/ub_truck.gltf,100,75,44.274911,47.581942,-122.167797,0,46.9369,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
4,1,truck,/veroviz/models/ub_truck.gltf,100,75,46.936868,47.581936,-122.168421,0,51.2123,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
5,1,truck,/veroviz/models/ub_truck.gltf,100,75,51.212269,47.581953,-122.169423,0,61.5737,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
6,1,truck,/veroviz/models/ub_truck.gltf,100,75,61.573713,47.581327,-122.171669,0,79.7666,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
7,1,truck,/veroviz/models/ub_truck.gltf,100,75,79.766560,47.580502,-122.175756,0,99.5076,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
8,1,truck,/veroviz/models/ub_truck.gltf,100,75,99.507580,47.580135,-122.180352,0,144.724,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
9,1,truck,/veroviz/models/ub_truck.gltf,100,75,144.724330,47.579604,-122.190923,0,149.875,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8


In [33]:
# We can visualize the 2D components of the assignments
# in a Leaflet map.  Note that the truck route along the 
# road is now displayed.  The drone routes don't look 
# much different from the previous "myArcs" dataframe, 
# as Leaflet is unable to represent the vertical components 
# of the drones' flight.
vrv.createLeaflet(arcs = assignmentsDF, nodes = myNodes, useArrows = False)

## Generating Cesium Dynamic Visualization

Now that we have the detailed "assignments" dataframe, we can generate a time-dynamic movie of the vehicle routes on a 3D map.

- See https://veroviz.org/docs/veroviz.createCesium.html for details on the function that makes this happen.
- See https://veroviz.org/documentation.html for instructions for viewing your Cesium movie.
- An example Cesium movie (for a different problem) is available here (no installation required):  https://veroviz.org/cesiumdemo.html

---

Before we continue, we need to know where Cesium was installed.
- One way to do specify Cesium's installation path is via system environment variables.  See https://veroviz.org/documentation.html for instructions.
- Alternatively, you can hard-code the path below.  On my Mac, I installed Cesium to `/Users/me/cesium`.  


In [34]:
# Specify the location where Cesium is installed on your computer:

# OPTION 1 -- Hard-coding the path here:
CESIUM_DIR = 'Users/me/cesium'

# OPTION 2 -- Using a system environment variable:
import os
CESIUM_DIR = os.environ['CESIUMDIR']

In [ ]:
import os
vrv.createCesium(assignments = assignmentsDF, 
                 nodes       = myNodes, 
                 startDate   = '2019-10-20', 
                 startTime   = '08:00:00', 
                 postBuffer  = 30, 
                 cesiumDir   = CESIUM_DIR, 
                 problemDir  = 'INFORMS_demo')

## Import/Export 

VeRoViz provides some utility functions to make it easy to save data you generated, and then import it later.  A few examples are shown below.

In [ ]:
vrv.exportDataToCSV(timeSec, 'time.csv')
vrv.exportDataToCSV(distMeters, 'dist.csv')
vrv.exportDataToCSV(myNodes, 'nodes.csv')

# You may also wish to export your arcs and solutions.

In [ ]:
importedNodes = vrv.importDataframe('nodes.csv')
importedNodes

--- 

That brings us to the end of the material covered in the INFORMS 2019 talk.  There are loads of other features available in VeRoViz that just couldn't be covered in a 15-minute talk.  Here are some suggestions to help you learn more about VeRoViz:
- Experiment with the interactive "Sketch" web utility (https://veroviz.org/sketch.html).  You can create nodes and arcs via point-and-click, and export your creation into Python.
- Check out the installation guide:  https://veroviz.org/documentation.html 
- See some other Jupyter notebooks available in the Downloads section:  https://veroviz.org/downloads/ 
- Take a look at the extensive (perhaps overwhelming) documentation at https://veroviz.org/docs/.  Each VeRoViz function also includes examples.